In [1]:
import datetime
import os
import shutil
import tempfile

In [2]:
#starting spark session
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Second use case in pyspark').getOrCreate()

In [3]:
#finding today's and yesterday's date
today=datetime.date.today()
today_str=today.strftime("%d-%m-%Y")
print(today_str)
yesterday = today - datetime.timedelta(days=9)
yesterday_str=yesterday.strftime("%d-%m-%Y")
print(yesterday_str)

16-04-2020
07-04-2020


'\none_day_before_yesterday = yesterday - datetime.timedelta(days=1)\none_day_before_yesterday_str=one_day_before_yesterday.strftime("%d-%m-%Y")\nprint(one_day_before_yesterday_str)\n'

In [4]:
#making temporary folder
f  =  tempfile.TemporaryDirectory(dir  =  "/home/bluepi/Downloads/result_2")
f.name

'/home/bluepi/Downloads/result_2/tmpsmrq0ole'

In [7]:
#file which have to be fetched
t=os.listdir('/home/bluepi/Downloads/result_2/latest_product/')
for file in t:
        if file.endswith(".csv"):
            name=file
            
print(name)

part-00000-dc9c1bff-067d-49d9-9304-9ce97073fab1-c000.csv


In [56]:
#copying file which have to be read
path=shutil.copy2(os.path.join('/home/bluepi/Downloads/result_2/latest_product/',name),f.name)
print(path)

/home/bluepi/Downloads/result_2/tmpl_1er0ju/part-00000-d42eda1d-2286-47a9-b456-b776c4551e34-c000.csv


In [57]:
#loading the main table
try:
    main_table=spark.read.csv(path,header=True)
except:
    print("File does not exist")

In [58]:
#showcasing the contents of table and counting the number of records
main_table.show(10)
print("Total rows in main_table=",main_table.count())

+----+-----------+-----+--------------------+
|p_id|     p_name|price|      Date_timestamp|
+----+-----------+-----+--------------------+
|   1|       Rank|  956|2020-01-20T14:39:...|
|   2|Mat Lam Tam|  556|2020-02-05T02:44:...|
|   3|   Greenlam|   17|2020-02-07T03:31:...|
|   4|         It|   55|2020-01-25T17:10:...|
|   5|  Holdlamis|  186|2020-02-21T04:16:...|
|   6|     Latlux|  181|2020-01-13T20:25:...|
|   7|   Greenlam|  294|2020-02-28T09:00:...|
|   8|   Alphazap|  739|2020-01-17T04:09:...|
|   9|   Alphazap|  423|2020-02-26T14:30:...|
|  10|   Home Ing|  681|2020-04-04T21:40:...|
+----+-----------+-----+--------------------+
only showing top 10 rows

Total rows in main_table= 104


In [59]:
#checking the schema
main_table.printSchema()

#updating the datatpe of columns
mt1 = main_table.withColumn("p_id",main_table['p_id'].cast("int"))\
                .withColumn("price",main_table['price'].cast("int"))\
                .withColumn("Date_timestamp", main_table['Date_timestamp'].cast("timestamp"))

#checking the schema
mt1.printSchema()

root
 |-- p_id: string (nullable = true)
 |-- p_name: string (nullable = true)
 |-- price: string (nullable = true)
 |-- Date_timestamp: string (nullable = true)

root
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- Date_timestamp: timestamp (nullable = true)



In [60]:
#making path as a variable
folder_path='/home/bluepi/Downloads/use_case_2/'
path=folder_path+yesterday_str

In [61]:
#reading the files from the folder

try:
    changes_to_be_done=spark.read.csv(path,header=True)
except:
    print("Folder does not exist")

In [62]:
#showcasing the contents of table
changes_to_be_done.show(10)
changes_to_be_done.groupBy('record_type').count().show()

+----+-----------+-----+-------------------+-----------+
|p_id|     p_name|price|     date_timestamp|record_type|
+----+-----------+-----+-------------------+-----------+
|  13|    Konklux|  866|2020-04-05 04:00:18|          U|
| 198|    Cookley|  304|2020-04-05 03:35:35|          U|
|  55|      Opela|   50|2020-04-05 03:15:21|          U|
|  14|Stringtough|  199|2020-04-05 03:54:35|          U|
|  87| Trippledex|   48|2020-04-05 03:43:00|          U|
| 175|    Fixflex|  498|2020-04-05 03:25:24|          I|
| 168|     Zathin|  259|2020-04-05 03:47:55|          I|
|  47|     Sonair|  547|2020-04-05 03:12:40|          D|
|  50|    Matsoft|  963|2020-04-05 03:20:45|          D|
|  18|  Cardguard| 1066|2020-04-05 21:25:10|          U|
+----+-----------+-----+-------------------+-----------+
only showing top 10 rows

+-----------+-----+
|record_type|count|
+-----------+-----+
|          U|   17|
|          D|    8|
|          I|    7|
+-----------+-----+



In [63]:
#checking the schema
changes_to_be_done.printSchema()

#updating the datatpe
ctbd1 = changes_to_be_done.withColumn("p_id",changes_to_be_done['p_id'].cast("int"))\
                          .withColumn("price",changes_to_be_done['price'].cast("int"))\
                          .withColumn("Date_timestamp", changes_to_be_done['Date_timestamp'].cast("timestamp"))

#checking the schema
ctbd1.printSchema()

root
 |-- p_id: string (nullable = true)
 |-- p_name: string (nullable = true)
 |-- price: string (nullable = true)
 |-- date_timestamp: string (nullable = true)
 |-- record_type: string (nullable = true)

root
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- Date_timestamp: timestamp (nullable = true)
 |-- record_type: string (nullable = true)



In [64]:
#registering the dataframe in tables

mt1.registerTempTable('Product_table')
ctbd1.registerTempTable('Changes_table')

In [65]:
#for inserting the data

#to retrieve the latest data from the data to be inserted
table=spark.sql(""" select c1.p_id,c1.p_name,c1.price,c1.Date_timestamp
                    from Changes_table as c1
                    inner join
                    (
                    select p_id, max(Date_timestamp) as Datetime
                    from Changes_table
                    where record_type='I'
                    group by p_id
                    ) as c2
                    on c1.p_id=c2.p_id and c1.Date_timestamp=c2.Datetime
                    order by c1.p_id""")

#converting dataframe into table
table.registerTempTable('Table')


#to insert data
table1=spark.sql(""" select * 
                    from Product_table
                    union
                    select * 
                    from Table""")


#converting dataframe with inserted values into table
table1.registerTempTable('Data_after_insertion')


#printing the details
print("Total inserted=",table.count())
print("Total rows in main table after inserting=",table1.count())
print("New inserted id's=",[int(row.p_id) for row in table.collect()])

Total inserted= 7
Total rows in main table after inserting= 111
New inserted id's= [102, 148, 158, 168, 175, 185, 198]


In [66]:
#for updating the data

#to retrieve the latest data from the data to be updated
table=spark.sql(""" select c1.p_id,c1.p_name,c1.price,c1.Date_timestamp
                    from Changes_table as c1
                    inner join
                    (
                    select p_id, max(Date_timestamp) as Datetime
                    from Changes_table
                    where record_type='U'
                    group by p_id
                    ) as c2
                    on c1.p_id=c2.p_id and c1.Date_timestamp=c2.Datetime
                    order by c1.p_id""")

#converting dataframe into table
table.registerTempTable('Table')


#to update data
table1=spark.sql(""" select *
                   from
                   (
                    select *
                    from Data_after_insertion 
                    where p_id not in(select p_id from Table)
                    union
                    select *
                    from Table
                    ) as ut
                    order by ut.p_id""")


#converting dataframe with deleted values into table
table1.registerTempTable('Data_after_updation')


#extracting historical data from the Data_after_insertion table which has been updated
history_of_updated_products = spark.sql(""" select d.p_id,d.p_name,d.price,d.Date_timestamp
                                            from Data_after_insertion as d
                                            left join Table t
                                            on d.p_id=t.p_id
                                            order by d.p_id""")


#printing the details
print("Total updated=",table.count())
print("Total rows in main table after updating=",table1.count())
print("New updated id's=",[int(row.p_id) for row in table.collect()])

Total updated= 16
Total rows in main table after updating= 111
New updated id's= [2, 12, 13, 14, 18, 32, 37, 55, 76, 79, 87, 90, 140, 153, 174, 198]


In [67]:
#for deleting the data

#to retrieve the latest data from the data to be deleted
table=spark.sql(""" select c1.p_id,c1.p_name,c1.price,c1.Date_timestamp
                    from Changes_table as c1
                    inner join
                    (
                    select p_id, max(Date_timestamp) as Datetime
                    from Changes_table
                    where record_type='D'
                    group by p_id
                    ) as c2
                    on c1.p_id=c2.p_id and c1.Date_timestamp=c2.Datetime
                    order by c1.p_id""")

#converting dataframe into table
table.registerTempTable('Table')


#to delete data
Final_latest_table=spark.sql(""" select * 
                    from Data_after_updation 
                    where p_id  not in(select p_id
                                         from Table)""")


#converting dataframe with deleted values into table
Final_latest_table.registerTempTable('Data_after_deletion')


#extracting data from the Data_after_insertion table which has been deleted
deleted_data=spark.sql(""" select * 
                    from Data_after_insertion 
                    where p_id  in (select p_id
                                    from Table)
                    order by p_id""")

#printing the details
print("Total deleted=",table.count())
print("Total rows in main table after deleted=",Final_latest_table.count())
print("Deleted id's=",[int(row.p_id) for row in table.collect()])


Total deleted= 8
Total rows in main table after deleted= 103
Deleted id's= [23, 31, 36, 47, 50, 71, 76, 97]


In [68]:
#merging the historical data

product_history = history_of_updated_products.union(deleted_data)

In [69]:
#specifying path to export products history and new changes in the main table
path_for_product_history='/home/bluepi/Downloads/result_2/product_history/'
path_for_latest_product='/home/bluepi/Downloads/result_2/latest_product/'

In [70]:
#exporting the HISTORICAL data into csv files 

product_history.repartition(1).write.csv(path_for_product_history+yesterday_str, header=True)

In [71]:
#exporting the FINAL TABLE

Final_latest_table.repartition(1).write.csv(path_for_latest_product, header=True,mode='overwrite')

In [72]:
#deleting the temp folder
f.cleanup()